In [1]:
import numpy as np
import os

# plotting 
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import PyPDF2

# data manager and analysis
import vodex as vx
import numan as nu

# writing files
import tifffile as tif

%load_ext autoreload
%autoreload 2

# Project structure: 

Provide the project folder with the "processed" folder created in the previous notebook. 

As you keep going with the analysis, the folder will have the following structure: 

```
processed
│   experiment.json <----------------------------------- (DONE in 01) the file that contains everything about the experiment, you are creating it once at the beginning of the processing and reusing ever after
│   experiment_dff.json <------------------------------- (DONE in 01) everything about the experiment, but loads from the dff movie, not from the raw data
└───dff_movie  <---------------------------------------- (DONE in 01) the dff movie :) 
│   │   dff_movie_0000.tif
│   │   dff_movie_0001.tif
│   │   ... 
└───tscore_volumes  <----------------------------------- (DONE in 02) t-score tif files per pair
│   └───2v3
│       │   tscore_2v3.tif
│   └───3v5
│       │   tscore_3v5.tif
│   └───2v5
│       │   tscore_2v5.tif
│   └───2vB
│       │   tscore_2vB.tif
│   └───3vB
│       │   tscore_3vB.tif
│   └───5vB
│       │   tscore_5vB.tif
│   └───BvB1
│       │   tscore_BvB1.tif
│   └───BvB2
│       │   tscore_BvB2.tif
│   └───BvB3
│       │   tscore_BvB3.tif
└───diff_volumes  <------------------------------------- (DONE in 02) absolute difference tif files per pair
│   └───2v3
│       │   diff_2v3.tif
│   └───3v5
│       │   diff_3v5.tif
│   └───...
└───spots
│   └───imaris  <--------------------------------------- (DONE after 02) ATTENTION : You need to put stuff generated by imaris into this folder!!! 
│       │   └───tscore_2v3_Statistics
│       │       │     tscore_2v3_Position.csv
│       │       │     tscore_2v3_Diameter.csv
│       │       │     ...
│       │   └───tscore_3v5_Statistics
│       │       │     tscore_3v5_Position.csv
│       │       │     tscore_3v5_Diameter.csv
│       │       │     ...
│       │   └───tscore_2v5_Statistics
│       │       │     ...
│       │   └───...
│   └───signals  <-------------------------------------- (DONE in 03, WILL BE UPDATED in this notebook) json files with the extracted signals, also will have the group info 
│       │   spots_2v3.json
│       │   spots_3v5.json
│       │   spots_2v5.json
│       │     ...
│       └───reports  <---------------------------------- tiffs and pdf with the cells significant in any pairwise comparison
│       │   └───all_significant  <---------------------- tiffs and pdf with all significant cells per group
│       │       │   └───signals  <---------------------- pdfs with signals
│       │       │       │     ...
│       │       │   └───images <------------------------ tif masks 
│       │       │       │     ...
│       │   └───groupped  <----------------------------- tiffs and pdf where the cells are groupped based on signal shape .. or anything else you want
│       │       │   readme.txt  <----------------------- ATTENTION : you need to describe the groups
│       │       │   └───signals  <---------------------- pdfs with signals
│       │       │       │     ...
│       │       │   └───images  <----------------------- tif masks 
│       │       │       │     ...
```

# Set project folder

The processed/spots/signals should already exist and have the extracted signals saved in there. 

In [2]:
project_folder = "D:/Code/repos/numan/notebooks/data/2vs3vs5/"
path = os.path.join(project_folder, 'processed')

assert os.path.isdir( os.path.join(path, "spots", "signals")), "the directory 'processed/spots/signals' doesn't exist in the project," \
                                " did you forget to run the previous notebook?"

os.chdir(path)
os.getcwd()

'D:\\Code\\repos\\numan\\notebooks\\data\\2vs3vs5\\processed'

# Load experiment with the raw data and define conditions: 

In [3]:
# don't forget to give the conditions names - they are used for plotting. 
blank = vx.Condition(vx.Stimulus('blank','blank'), name = 'blank')
dot2 = vx.Condition(vx.Stimulus('2dot','2dot'), name = 'dot2')
dot3 = vx.Condition(vx.Stimulus('3dot','3dot'), name = 'dot3')
dot5 = vx.Condition(vx.Stimulus('5dot','5dot'), name = 'dot5')

experiment = vx.from_json(vx.Experiment,'experiment.json')
experiment.summary()

Total of 8 files.
Check the order :
[ 0 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0.ome.tif : 8910 frames
[ 1 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_1.ome.tif : 8909 frames
[ 2 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_2.ome.tif : 8909 frames
[ 3 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_3.ome.tif : 8909 frames
[ 4 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_4.ome.tif : 8909 frames
[ 5 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_5.ome.tif : 8909 frames
[ 6 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_6.ome.tif : 8909 frames
[ 7 ] 20220421_ok08_abtl_h2bcamp6s_9dpf_2v3v5_2P_1_MMStack_Pos0_7.ome.tif : 4092 frames

Cycle length: 3692
Condition ['blank']: for 364 frames
Condition ['3dot']: for 52 frames
Condition ['blank']: for 260 frames
Condition ['2dot']: for 52 frames
Condition ['blank']: for 312 frames
Condition ['5dot']: for 52 frames
Condition ['blank']: for 312 fra

## get the volume ids for the conditions that you will be comparing : 
This is the same as the ones we got in the notebook 02, under " get the volumes corresponding to different conditions "

In [4]:
def merge_idx(idx_list):
    return np.sort(np.concatenate(idx_list))

# indeces of the volumes to load
dot2_idx = experiment.select_volumes(dot2)
dot3_idx = experiment.select_volumes(dot3)
dot5_idx = experiment.select_volumes(dot5)
# as blank, we will take the blanks 1 volume before the stimuli
blank2_idx = dot2_idx - 1
blank3_idx = dot3_idx - 1
blank5_idx = dot5_idx - 1

print(f"We found:\n", 
      f"{dot2_idx.shape} volumes with 2dot")
print(f" {dot3_idx.shape} volumes with 3dot")
print(f" {dot5_idx.shape} volumes with 5dot")

We found:
 (54,) volumes with 2dot
 (54,) volumes with 3dot
 (54,) volumes with 5dot


Get wrapper functions ( just run it )

In [10]:
def get_p_and_control(sa, group1_idx, group2_idx, name_p, vs_blank = False):
    """
    Given SignalAnalyzer and two groups of volume idx, 
    Calculates the p-values for the difference of means for the two groups  (p) 
    and for one volume right before them (p_control). 
    """
    print( f"Getting p-values for {name_p}")
    p = sa.get_p_list_of_difference_of_means(group1_idx, group2_idx)
    
    if vs_blank:
        p_control = None
    else:
        p_control = sa.get_p_list_of_difference_of_means(group1_idx-1, group2_idx-1)
    return p, p_control

def get_significant(p, p_control, vs_blank = False):
    """
    Figures out what spots have significant p values for the difference of the mean. 
    It it is for comparing the blanks, it ignores the control.
    """
    sig_pvalue = 0.05
    
    significant_cells_stim = np.array(p)<sig_pvalue
    n_stim = np.sum(significant_cells_stim)
    
    if vs_blank:
        significant_cells = significant_cells_stim
        n_significant = n_stim
        n_control = "NA"
    else:
        significant_cells_control = np.array(p_control)<sig_pvalue
        n_control = np.sum(significant_cells_control)

        significant_cells = np.logical_and(significant_cells_stim,~significant_cells_control)
        n_significant = np.sum(significant_cells)
        
    print(f'Significant: {n_significant} | stim only : {n_stim} | control only : {n_control} ')
    
    return significant_cells

def p_value_histogram(p, p_control, ax, title, vs_blank = False):
    
    bins = np.arange(0,1.2,0.05)
    ax.hist(p, bins, alpha=0.5, label='stimulus')
    if not vs_blank:
        ax.hist(p_control, bins, alpha=0.5, label='control')
    ax.legend(loc='upper right')
    ax.set_title(title)
    
    
def find_significant_spots(spot_tag):
    
    spots = nu.Spots.from_json(f"spots/signals/spots_{spot_tag}.json")
    
    # get dff for each cell
    sliding_window = 15 # in volumes
    # initialise signal analyser with DFF spot signals
    sa = nu.SignalAnalyzer(spots.signals.as_dff(sliding_window))

    # calculate p-values using bootstrap
    p2v5, p2v5_c = get_p_and_control(sa, dot2_idx, dot5_idx, "2v5")
    sig2v5 = get_significant(p2v5, p2v5_c)
    p3v5, p3v5_c = get_p_and_control(sa, dot3_idx, dot5_idx, "3v5")
    sig3v5 = get_significant(p3v5, p3v5_c)
    p2v3, p2v3_c = get_p_and_control(sa, dot2_idx, dot3_idx, "2v3")
    sig2v3 = get_significant(p2v3, p2v3_c)
    
    # We are not considering the first presentation of "2", 
    # because there is no info on the blank right before it  ( thus the [1:] ) 
    p2vB, p2vB_c = get_p_and_control(sa, dot2_idx[1:], blank2_idx[1:], "2vB", vs_blank = True)
    sig2vB = get_significant(p2vB, p2vB_c, vs_blank = True)
    p3vB, p3vB_c = get_p_and_control(sa, dot3_idx, blank3_idx, "3vB", vs_blank = True)
    sig3vB = get_significant(p3vB, p3vB_c, vs_blank = True)
    p5vB, p5vB_c = get_p_and_control(sa, dot5_idx, blank5_idx, "5vB", vs_blank = True)
    sig5vB = get_significant(p5vB, p5vB_c, vs_blank = True)

    # show histograms
    fig, ax = plt.subplots(2, 3, figsize=(12,6), dpi=160)
    ax = ax.flatten()
    fig.suptitle(f"P-values of spots from the T-score image for {spot_tag}")

    p_value_histogram(p2v5, p2v5_c, ax[0], '2v5')
    p_value_histogram(p3v5, p3v5_c, ax[1], '3v5')
    p_value_histogram(p2v3, p2v3_c, ax[2], '2v3')
    p_value_histogram(p2vB, p2vB_c, ax[3], '2vB', vs_blank = True)
    p_value_histogram(p3vB, p3vB_c, ax[4], '3vB', vs_blank = True)
    p_value_histogram(p5vB, p5vB_c, ax[5], '5vB', vs_blank = True)
    
    # add the p-values and significance info to the spots
    spots.add_groups({"p2v5":p2v5, "p2v5_c":p2v5_c, "sig2v5":sig2v5,
                      "p3v5":p3v5, "p3v5_c":p3v5_c, "sig3v5":sig3v5,
                      "p2v3":p2v3, "p2v3_c":p2v3_c, "sig2v3":sig2v3,

                      "p2vB":p2vB, "p2vB_c":p2vB_c, "sig2vB":sig2vB,
                      "p3vB":p3vB, "p3vB_c":p3vB_c, "sig3vB":sig3vB,
                      "p5vB":p5vB, "p5vB_c":p5vB_c, "sig5vB":sig5vB})

    spots.to_json(f"spots/signals/spots_{spot_tag}.json")


# Find significant cells and save them 
It will find the p-vlaues for different stimuli pairs and will add this information into the existing files with the spot signals in processed/spots/signals. Those files will be updated, no new files will be created. 

```
processed
└───spots
│   └───signals  <-------------------------------------- (DONE in 03) json files with the extracted signals, also will have the group info after you added it
│       │   spots_2v3.json
│       │   spots_3v5.json
│       │   spots_2v5.json
│       │     ...
```

In [ ]:
np.random.seed(42)

tag = "2v5"
print(f"Analysing cells from T-score image for {tag}_________________________________________________________")
find_significant_spots(tag)
tag = "2v3"
print(f"Analysing cells from T-score image for {tag}_________________________________________________________")
find_significant_spots(tag)
tag = "3v5"
print(f"Analysing cells from T-score image for {tag}_________________________________________________________")
find_significant_spots(tag)
tag = "2vB"
print(f"Analysing cells from T-score image for {tag}_________________________________________________________")
find_significant_spots(tag)
tag = "3vB"
print(f"Analysing cells from T-score image for {tag}_________________________________________________________")
find_significant_spots(tag)
tag = "5vB"
print(f"Analysing cells from T-score image for {tag}_________________________________________________________")
find_significant_spots(tag)
